In [12]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time
from tqdm import tqdm

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [2]:
# Load the gridded FRP data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_gridstats')
print(f"Number of grid cells: {grid.size().getInfo()}")
grid = grid.select(['grid_index','Fire_ID'])
print(grid.first().propertyNames().getInfo())

Number of grid cells: 57232
['system:index', 'grid_index', 'Fire_ID']


In [3]:
# CBI Asset folder (replace with your asset path)
cbidir = 'projects/jfsp-aspen/assets/CBI/'
# List all CBI images in the folder
cbi_images = ee.data.listAssets({'parent': cbidir})['assets']
cbi_image_list = [
    ee.Image(f['id']) for f in cbi_images if f['id'].endswith('CBI_bc')
]
print(len(cbi_image_list))

100


In [4]:
# Setup seperate functions for average/stdev and extremes (percentiles)
def calculate_average(cbi_image, grid_fc, fire_id):
    """Calculate mean and standard deviation of CBI within each grid cell."""
    return cbi_image.reduceRegions(
        collection=grid_fc,
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(), sharedInputs=True
        ),
        scale=30
    ).map(lambda f: f.set("Fire_ID", fire_id))

def calculate_extremes(cbi_image, grid_fc, fire_id):
    """Calculate percentiles (90th, 95th, 99th) of CBI within each grid cell."""
    return cbi_image.reduceRegions(
        collection=grid_fc,
        reducer=ee.Reducer.percentile([90, 95, 99]),
        scale=30
    ).map(lambda f: f.set("Fire_ID", fire_id))

def merge_collections(primary, secondary, join_field="grid_index"):
    """Join two FeatureCollections and merge their properties."""
    join = ee.Join.inner()
    filter_condition = ee.Filter.equals(
        leftField=join_field, 
        rightField=join_field
    )
    joined = join.apply(primary, secondary, filter_condition)

    # Flatten joined results
    def merge_features(f):
        primary_props = ee.Feature(f.get("primary")).toDictionary()
        secondary_props = ee.Feature(f.get("secondary")).toDictionary()
        combined_props = primary_props.combine(secondary_props)
        return ee.Feature(None, combined_props)

    return joined.map(merge_features)

print("Functions ready !")

Functions ready !


In [7]:
print(cbi_image_list[0].get('system:index').getInfo())

None


In [9]:
cbi_image_list[1]

In [13]:
# Initialize export tasks for all fires
# Initialize an empty collection to hold all results
all_fires_stats = ee.FeatureCollection([])
for cbi_image in tqdm(cbi_image_list, desc="Processing Fires"):
    
    # Extract Fire ID from the image file name
    fire_id = cbi_image.get('fireID').getInfo()
    # Filter grid cells for this fire
    fire_grid = grid.filter(ee.Filter.eq("Fire_ID", fire_id))

    # Calculate mean/stdDev and percentiles
    average_stats = calculate_average(cbi_image, fire_grid, fire_id)
    extremes_stats = calculate_extremes(cbi_image, fire_grid, fire_id)

    # Merge results
    stats = merge_collections(average_stats, extremes_stats)

    # Add to the global collection
    all_fires_stats = all_fires_stats.merge(stats)

# export table to Drive.
export_task = ee.batch.Export.table.toDrive(
    collection=all_fires_stats,
    description='gridstats_cbibc',
    fileNamePrefix='gridstats_cbibc',
    fileFormat='CSV', 
    folder='CBI'
)

export_task.start()
print("Export to Earth Engine Asset started!")
monitor_export(export_task, 120)

Processing Fires: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.16it/s]


Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo